In [1]:
import mysql.connector
from mysql.connector import errorcode
import pandas as pd
import numpy as np

config = {
  'user': 'root',
  'password': '',
  'host': 'localhost',
  'database': 'dualt_ticino',
  'raise_on_warnings': True
}

try:
    cnx = mysql.connector.connect(**config)
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)

import mysql.connector

data_folder = "../data/"


def fetch_table_data(table_name):
    
    cursor = cnx.cursor()
    cursor.execute('select * from ' + table_name)

    header = [row[0] for row in cursor.description]

    rows = cursor.fetchall()

    # Closing connection
    cnx.close()

    return header, rows


def export(table_name):
    header, rows = fetch_table_data(table_name)

    # Create csv file
    f = open(table_name + '.csv', 'w')

    # Write header
    f.write(','.join(header) + '\n')

    for row in rows:
        f.write(','.join(str(r) for r in row) + '\n')

    f.close()
    print(str(len(rows)) + ' rows written successfully to ' + f.name)


# Tables to be exported
#export('TABLE_1')
#export('TABLE_2')

In [4]:
cursor = cnx.cursor()

query = ("SELECT no_idObject as ac_activity, no_sender, no_recipient, "
             "no_type, no_timestamp, no_isRead, au_evaluation, tr.ar_evaluation "
         "FROM notifications AS n "
             "RIGHT JOIN activities_users ON n.no_idObject = activities_users.ac_activity "
             "RIGHT JOIN users ON activities_users.us_user = users.us_user "
             "RIGHT JOIN activities_responsables AS tr ON tr.ac_activity = n.no_idObject "
         "WHERE no_type = 'feedback response' OR no_type='feedback request' "
         "ORDER BY no_timestamp ASC "
        )

cursor.execute(query)

activities = {}
for (ac_activity, no_sender, no_recipient, no_type, no_timestamp, no_isRead, au_evaluation, ar_evaluation) in cursor:
    ac_activity = int(ac_activity)
    if not activities.get(ac_activity):
        activities[ac_activity] = {
            'requests':[],
            'responses':[], 
            'edits': [],
            'sender':None,
            'recipient':None,
            'supervisor_grade':None,
            'student_grade':None
        }
    
    action = 'responses'
    recipient = no_sender
    sender = no_recipient
    if no_type == 'feedback request':
        action = 'requests'
        recipient = no_recipient
        sender = no_sender
        
    activities[ac_activity]['sender'] = sender
    activities[ac_activity]['recipient'] = recipient
    activities[ac_activity]['student_grade'] = au_evaluation
    activities[ac_activity]['supervisor_grade'] = ar_evaluation
        
    activities[ac_activity][action].append(no_timestamp)

cursor.close()

SELECT no_idObject as ac_activity, no_sender, no_recipient, no_type, no_timestamp, no_isRead, au_evaluation, tr.ar_evaluation FROM notifications AS n RIGHT JOIN activities_users ON n.no_idObject = activities_users.ac_activity RIGHT JOIN users ON activities_users.us_user = users.us_user RIGHT JOIN activities_responsables AS tr ON tr.ac_activity = n.no_idObject WHERE no_type = 'feedback response' OR no_type='feedback request' ORDER BY no_timestamp ASC 


True

In [6]:
cursor = cnx.cursor()

query = ("""
        SELECT ac_activity, data 
        FROM LOG_activities 
         WHERE operazione='U'
         """)

cursor.execute(query)

for (ac_activity, data) in cursor:
    if not activities.get(ac_activity):
        continue
        
    activities[ac_activity]['edits'].append(data)

cursor.close()

True

In [8]:
activities_feedbacks_info = []
for activity,v in activities.items():
    requests    = v['requests']
    responses   = v['responses']
    edits       = v['edits']
    sender      = v['sender']
    recipient   = v['recipient']
    student_grade    = v['student_grade']
    supervisor_grade = v['supervisor_grade']
    end_date  = None
    no_end    = False
    
    for request_date in requests:
        start_date = request_date
        if no_end or (end_date and request_date < end_date):
            continue
        end_date = None
        no_end   = True
        for response_date in responses:
            if response_date < request_date:
                continue
            no_end   = False
            end_date = response_date
            break
        
        request_times = 1
        for _request_date in requests:
            if _request_date > start_date and (no_end or _request_date < end_date):
                request_times+=1
                
        delay_hours = None
        if end_date:
            delay_hours = round((end_date-start_date).total_seconds()/60/60)
            
        edits_between = 0
        edits_after   = 0
        if end_date:
            next_response_date = None
            
            for response_date in responses:
                if response_date <= end_date:
                    continue
                next_response_date = response_date
                break
            
            
            for edit_date in edits:
                if edit_date > start_date and edit_date < end_date:
                    edits_between += 1
                if edit_date > end_date and (not next_response_date or edit_date < next_response_date):
                    edits_after += 1
                
            
        activity_info = {
                         'ac_activity':activity, 'sender': sender, 'recipient': recipient,
                         'request_date':start_date,'response_date':end_date, 'delay_hours':delay_hours,
                         'times_before_answer':request_times,
                         'edits_between': edits_between, 'edits_after': edits_after,
                         'student_grade': student_grade, 'supervisor_grade':supervisor_grade
                        }
        
        activities_feedbacks_info.append(activity_info)

In [10]:
cursor = cnx.cursor()

try:
    cursor.execute("DROP TABLE IF EXISTS activities_feedbacks_info; ");
except Exception as e:
    print(e)

query = (
         "CREATE TABLE activities_feedbacks_info "
         "     ( ac_activity INT(11) NOT NULL , "
         "       sender INT(11) NOT NULL , "
         "       recipient INT(11) NOT NULL , "
         "       request_date TIMESTAMP NOT NULL , "
         "       response_date TIMESTAMP NULL DEFAULT NULL , "
         "       delay_hours INT(5) NULL DEFAULT NULL , "
         "       edits_between INT(3) NULL DEFAULT NULL , "
         "       edits_after INT(3) NULL DEFAULT NULL , "
         "       times_before_answer INT(3) NOT NULL DEFAULT '1', "
         "       student_grade DECIMAL(3,2) NULL DEFAULT NULL , " 
         "       supervisor_grade DECIMAL(3,2) NULL DEFAULT NULL , " 
         "INDEX (ac_activity)) ENGINE = InnoDB;"
        )

cursor.execute(query);

1051: Unknown table 'dualt_ticino.activities_feedbacks_info'


In [11]:
cursor.executemany("""
    INSERT INTO activities_feedbacks_info 
        (ac_activity, sender, recipient, 
            request_date, response_date, 
            delay_hours, edits_between, 
            edits_after, times_before_answer,
            student_grade, supervisor_grade
        )
    VALUES (
        %(ac_activity)s, %(sender)s, %(recipient)s, 
        %(request_date)s, %(response_date)s, %(delay_hours)s, 
        %(edits_between)s, %(edits_after)s, %(times_before_answer)s,
        %(student_grade)s, %(supervisor_grade)s)""", activities_feedbacks_info)

cursor.close()

True

In [12]:
cnx.commit() 

In [13]:
query = ("SELECT info.*, a.activity_school_year, a.start_year "
         "FROM activities_feedbacks_info AS info "
         "NATURAL JOIN activities_users "
         "NATURAL LEFT JOIN V_all_activities_users AS a "
         "ORDER BY info.request_date"
        )
results = pd.read_sql_query(query, cnx)

In [14]:
results.to_csv(f"{data_folder}activities_feedbacks_info.csv", index=False)

In [21]:
query = ("SELECT t1.*,t2.sent FROM "
         "(SELECT no_recipient AS us_user, COUNT(*) AS received FROM `notifications` WHERE no_type = 'feedback request' GROUP BY no_recipient) AS t1 "
         "NATURAL LEFT JOIN (SELECT no_sender AS us_user,COUNT(*) AS sent FROM notifications WHERE no_type = 'feedback response' GROUP BY no_sender) AS t2 "
         "NATURAL JOIN users_userTypes "
         "WHERE ut_user_type = 'formatore'"
        )
results = pd.read_sql_query(query, cnx)
results.to_csv(f"{data_folder}supervisors_feedbacks.csv", index=False)

In [75]:
query = ("""SELECT month, activity_school_year, start_year, n_activities, n_recipes, n_experiences,
                ROUND(AVG(n_user_activities),2) AS avg_n_user_activities,
                ROUND(AVG(n_user_recipes),2) AS avg_n_user_recipes,
                ROUND(AVG(n_user_experiences),2) AS avg_n_user_experiences

            FROM (SELECT MONTH(ac_date) AS month, activity_school_year, start_year, us_user, COUNT(*) AS n_user_activities, 
                        SUM(at_activityType = 'recipe') AS n_user_recipes, 
                        SUM(at_activityType = 'experience') AS n_user_experiences 
                    FROM activities_users_school_year
                    GROUP BY us_user, MONTH(ac_date), activity_school_year, start_year) AS activities_per_user
            NATURAL LEFT JOIN (
                SELECT MONTH(ac_date) AS month, activity_school_year, start_year, COUNT(*) AS n_activities, 
                    SUM(at_activityType = 'recipe') AS n_recipes, 
                    SUM(at_activityType = 'experience') AS n_experiences 
                FROM activities_users_school_year
                GROUP BY MONTH(ac_date), activity_school_year, start_year
            ) AS n_act
            WHERE activity_school_year <= 3
            GROUP BY month, activity_school_year, start_year
""")
results = pd.read_sql_query(query, cnx)

In [76]:
results.to_csv(f"{data_folder}years_test.csv", index=False)

In [77]:
cnx.close()